In [1]:
import selenium
from selenium import webdriver
import time
import requests
import os
from PIL import Image
import io
import hashlib
# This is the path I use
#DRIVER_PATH = '/Users/anand/Desktop/chromedriver'
# Put the path for your ChromeDriver here
DRIVER_PATH = 'C:/Users/user/Melza/UTS/chromedriver/chromedriver.exe'


def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls    
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(30)
            return
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls

def persist_image(folder_path:str,file_name:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        folder_path = os.path.join(folder_path,file_name)
        if os.path.exists(folder_path):
            file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        else:
            os.mkdir(folder_path)
            file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=3000)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

# No.1 Image Scrapping

In [16]:
#if __name__ == '__main__':
    #wd = webdriver.Chrome(executable_path=DRIVER_PATH) 
    #queries = ["Mohammad Hatta"]  #change your set of querries here
    #for query in queries:
        #wd.get('https://google.com')
        #search_box = wd.find_element_by_css_selector('input.gLFyf')
        #search_box.send_keys(query)
        #links = fetch_image_urls(query, 22,wd)
        #images_path = '/Users/anand/Desktop/contri/images'  #enter your desired image path
        #images_path = 'C:/Users/user/Melza/UTS/hasil/tokoh'
        #for i in links:
            #persist_image(images_path,query,i)
    #wd.quit()

Found: 100 search results. Extracting links from 0:100
Found: 22 image links, done!
SUCCESS - saved https://asset-a.grid.id/crop/118x157:1978x2378/x/photo/2020/05/19/332787236.jpg - as C:/Users/user/Melza/UTS/hasil/tokoh\Mohammad Hatta\3b5f589681.jpg
SUCCESS - saved https://thumb.viva.co.id/media/frontend/tokoh/2017/08/16/59941b71bae54-mohammad-hatta2_216_287.jpg - as C:/Users/user/Melza/UTS/hasil/tokoh\Mohammad Hatta\719f39bf4b.jpg
SUCCESS - saved https://cdn1-production-images-kly.akamaized.net/VnIMC1o5GxWT-jy33-WptUc5VlM=/640x360/smart/filters:quality(75):strip_icc():format(jpeg)/kly-media-production/medias/1314465/original/051417900_1470904327-Banner_hatta_2.jpg - as C:/Users/user/Melza/UTS/hasil/tokoh\Mohammad Hatta\4b54ec548c.jpg
SUCCESS - saved https://asset.kompas.com/crops/QMtrhCUrN3x2DAQHnJ70FGD4DCc=/0x155:2590x1882/750x500/data/photo/2020/06/12/5ee39b8b9fed5.jpg - as C:/Users/user/Melza/UTS/hasil/tokoh\Mohammad Hatta\49932b83ff.jpg
SUCCESS - saved https://asset.kompas.com/cr

In [32]:
from os import listdir
import os.path
import re
import glob
import numpy as np
import shutil
import albumentations as A
import cv2

# No.2&3

In [33]:
import os.path
from os import listdir

def renameFile(path, depth=99):
    folder = os.listdir(path)
    index = 1
    for i in folder:
        old_name = path + '/' + i
        new_name = i
        new_name = re.sub('[^A-Za-z]+', ' ', new_name)
        new_name = new_name.lower()
        new_name = path + '/' + new_name
        os.rename(old_name, new_name)
        
        if os.path.isdir(new_name):
            renameFile(new_name, depth - 1)
        else:
            os.chdir(path)
            folder_name = os.path.basename(os.path.dirname(os.path.realpath(i)))
            file_name = os.path.dirname(new_name) + '/' + '{0:0=2d}'.format(index) + '_' + folder_name.replace(' ', '') + '.jpg'
            os.rename(new_name, file_name)
            index += 1
            
        print("File {} renamed!".format(i))

# No.4

In [34]:
import glob
import numpy as np
import shutil

def check(path):
    folder = os.listdir(path)
    newpath = path2 + '/' + 'unclassifiedFile'
    if not os.path.exists(newpath):
        os.makedirs(newpath)
    for i in folder:
        folder_path = path + '/' + i
        getFilePahlawan = glob.glob(str(folder_path)+'/*')
        for j, filename in enumerate(getFilePahlawan):
            print(filename)
            try:
                tempimage = Image.open(filename)
                tempimage.verify() #verify that it is, in fact an image
                image = Image.open(filename)
                image = np.array(image)
                img = Image.fromarray(image)
                img.mode
                filename = filename.replace("'","").replace('"','')
                img.save(filename)
                print('This is save file : {}'.format(filename))
            except (IOError, SyntaxError, img.mode != 'RGB'):
                print('Bad file:', filename) #print out the names of corrupt files
                shutil.move(filename, newpath + '/' + os.path.basename(filename))

# No.5

In [35]:
import albumentations as A
import cv2
from matplotlib import pyplot as plt
from PIL import Image

def overSampling(path):
    folder = os.listdir(path)
    for i in folder:
        folder_path = path + '/' + i
        getFilePahlawan = glob.glob(str(folder_path)+'/*')
        for j, filename in enumerate(getFilePahlawan):
            transform1 = A.Compose([
                 A.RandomBrightnessContrast(brightness_limit=0.5, contrast_limit=0.5, p=0.5)
                 ])
            image = cv2.imread(filename)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            transformed1 = transform1(image=image)
            transformed_image1 = transformed1["image"]
            result1 = cv2.imwrite(folder_path + '/' + "randombrightnesscontrast" + os.path.basename(filename), transformed_image1)
            
            transform2 = A.Compose([
                 A.Rotate(limit=90, always_apply=True, p=0.5)
                 ])
            transformed2 = transform2(image=image)
            transformed_image2 = transformed2["image"]
            result2 = cv2.imwrite(folder_path + '/' + "rotate" + os.path.basename(filename), transformed_image2)
        
            transform3 = A.Compose([
                 A.HueSaturationValue(hue_shift_limit=15, sat_shift_limit=20, val_shift_limit=20, p=0.5)
                 ])
            transformed3 = transform3(image=image)
            transformed_image3 = transformed3["image"]
            result3 = cv2.imwrite(folder_path + '/' + "hsv" + os.path.basename(filename), transformed_image3)


# No.6

In [36]:
def fileCount(path):
    folder = os.listdir(path)
    count = []
    for i in folder:
        folder_path = path + '/' + i
        getFilePahlawan = glob.glob(str(folder_path)+'/*')
        total_file = len(getFilePahlawan)
        count.append(total_file)
        
        if total_file < max(count):
            diff = max(count) - total_file
            take_file = getFilePahlawan[0:diff]
            for j, filename in enumerate(take_file):
                transform = A.Compose([
                    A.Flip(p=0.5)
                    ])
                image = cv2.imread(filename)
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                transformed = transform(image=image)
                transformed_image = transformed["image"]
                result = cv2.imwrite(folder_path + '/' + "flip" + os.path.basename(filename), transformed_image)

# No.7

In [37]:
def splitRGB(path):
    folder = os.listdir(path)
    for i in folder:
        folder_path = path + '/' + i
        getFilePahlawan = glob.glob(str(folder_path)+'/*')
        #membuatfolder
        red_path = folder_path + '/' + 'red'
        green_path = folder_path + '/' + 'green'
        blue_path = folder_path + '/' + 'blue'
        newpath = [red_path, green_path, blue_path]
        for k in newpath:
            if not os.path.exists(k):
                os.makedirs(k)
        for j, filename in enumerate(getFilePahlawan):
            img = Image.open(filename)
            pic = np.array(img)
            r = pic.copy()
            r[:, :, 0] = 0
            r[:, :, 1] = 0
            
            g = pic.copy()
            g[:, :, 0] = 0
            g[:, :, 2] = 0
            
            b = pic.copy()
            b[:, :, 1] = 0
            b[:, :, 2] = 0
            
            result_red = cv2.imwrite(red_path + "/" + "red" + "_" + os.path.basename(filename), r)
            result_green = cv2.imwrite(green_path + "/" + "green" + "_" + os.path.basename(filename), g)
            result_blue = cv2.imwrite(blue_path + "/" + "blue" + "_" + os.path.basename(filename), b)

# No.8

In [38]:
def diskritPixel(path):
    folder = os.listdir(path)
    for i in folder:
        folder_path = path + '/' + i
        folder_rgb = next(os.walk(folder_path))[1]
        for j in folder_rgb:
            rgb_path = folder_path + '/' + j
            getFileRGB = glob.glob(str(rgb_path)+'/*')
            convert_path = rgb_path + '/' + 'convert'
            if not os.path.exists(convert_path):
                os.makedirs(convert_path)
            for k, filename in enumerate(getFileRGB):
                img = Image.open(filename)
                pic = np.array(img)
                
                r = pic.copy()
                red_mask = r[:, :, 0] >= 180
                r[red_mask] = 225
                red_mask = r[:, :, 0] < 180
                r[red_mask] = 0
                
                g = pic.copy()
                green_mask =g[:, :, 1] >= 180
                g[green_mask] = 225
                green_mask = g[:, :, 1] < 180
                g[green_mask] = 0
                
                b = pic.copy()
                blue_mask = r[:, :, 2] >= 180
                b[blue_mask] = 225
                blue_mask = b[:, :, 2] < 180
                b[blue_mask] = 0
                
                result_red = cv2.imwrite(converth_path + "/" + "convert" + "_" + os.path.basename(filename), r)
                result_green = cv2.imwrite(converth_path + "/" + "convert" + "_" + os.path.basename(filename), g)
                result_blue = cv2.imwrite(converth_path + "/" + "convert" + "_" + os.path.basename(filename), b)

# No.9

In [39]:
def grayScale(path):
    folder = os.listdir(path)
    for i in folder:
        folder_path = path + '/' + i
        getFilePahlawan = glob.glob(str(folder_path)+'/*.jpg')
        gray_path = folder_path + '/' + 'grayscale'
        if not os.path.exists(gray_path):
            os.makedirs(gray_path)
        for j, filename in enumerate(getFilePahlawan):
            img = Image.open(filename)
            imgGray = img.convert('L')
            imgGray.save(gray_path + "/" + "gray" + "_" + os.path.basename(filename))

In [40]:
if __name__ == '__main__':
    path = 'C:/Users/user/Melza/UTS/hasil/tokoh'
    path2 = 'C:/Users/user/Melza/UTS/hasil'
    renameFile(path)
    check(path)
    overSampling(path)
    fileCount(path)
    splitRGB(path)
    diskritPixel(path)
    grayScale(path)
    print("BERHASILLL")

File 01_ahmaddahlan.jpg renamed!
File 02_ahmaddahlan.jpg renamed!
File 03_ahmaddahlan.jpg renamed!
File 04_ahmaddahlan.jpg renamed!
File 05_ahmaddahlan.jpg renamed!
File 06_ahmaddahlan.jpg renamed!
File 07_ahmaddahlan.jpg renamed!
File 08_ahmaddahlan.jpg renamed!
File 09_ahmaddahlan.jpg renamed!
File 10_ahmaddahlan.jpg renamed!
File 11_ahmaddahlan.jpg renamed!
File 12_ahmaddahlan.jpg renamed!
File 13_ahmaddahlan.jpg renamed!
File 14_ahmaddahlan.jpg renamed!
File 15_ahmaddahlan.jpg renamed!
File 16_ahmaddahlan.jpg renamed!
File 17_ahmaddahlan.jpg renamed!
File 18_ahmaddahlan.jpg renamed!
File 19_ahmaddahlan.jpg renamed!
File 20_ahmaddahlan.jpg renamed!
File 21_ahmaddahlan.jpg renamed!
File 22_ahmaddahlan.jpg renamed!
File 23_ahmaddahlan.jpg renamed!
File 24_ahmaddahlan.jpg renamed!
File 25_ahmaddahlan.jpg renamed!
File 26_ahmaddahlan.jpg renamed!
File 27_ahmaddahlan.jpg renamed!
File 28_ahmaddahlan.jpg renamed!
File 29_ahmaddahlan.jpg renamed!
File 30_ahmaddahlan.jpg renamed!
File 31_ah

File 43_red.jpg renamed!
File 44_red.jpg renamed!
File 45_red.jpg renamed!
File 46_red.jpg renamed!
File 47_red.jpg renamed!
File 48_red.jpg renamed!
File 49_red.jpg renamed!
File 50_red.jpg renamed!
File 51_red.jpg renamed!
File 52_red.jpg renamed!
File 53_red.jpg renamed!
File 54_red.jpg renamed!
File 55_red.jpg renamed!
File 56_red.jpg renamed!
File 57_red.jpg renamed!
File 58_red.jpg renamed!
File 59_red.jpg renamed!
File 60_red.jpg renamed!
File 61_red.jpg renamed!
File 62_red.jpg renamed!
File 63_red.jpg renamed!
File 64_red.jpg renamed!
File 65_red.jpg renamed!
File 66_red.jpg renamed!
File 67_red.jpg renamed!
File 68_red.jpg renamed!
File 69_red.jpg renamed!
File 70_red.jpg renamed!
File 71_red.jpg renamed!
File 72_red.jpg renamed!
File 73_red.jpg renamed!
File 74_red.jpg renamed!
File 75_red.jpg renamed!
File 76_red.jpg renamed!
File 77_red.jpg renamed!
File 78_red.jpg renamed!
File 79_red.jpg renamed!
File 80_red.jpg renamed!
File 81_red.jpg renamed!
File 82_red.jpg renamed!


File 77_green.jpg renamed!
File 78_green.jpg renamed!
File 79_green.jpg renamed!
File 80_green.jpg renamed!
File 81_green.jpg renamed!
File 82_green.jpg renamed!
File 83_green.jpg renamed!
File 84_green.jpg renamed!
File 85_green.jpg renamed!
File 86_green.jpg renamed!
File 87_green.jpg renamed!
File 88_green.jpg renamed!
File 89_green.jpg renamed!
File 90_green.jpg renamed!
File 91_green.jpg renamed!
File 92_green.jpg renamed!
File green renamed!
File 01_red.jpg renamed!
File 02_red.jpg renamed!
File 03_red.jpg renamed!
File 04_red.jpg renamed!
File 05_red.jpg renamed!
File 06_red.jpg renamed!
File 07_red.jpg renamed!
File 08_red.jpg renamed!
File 09_red.jpg renamed!
File 10_red.jpg renamed!
File 11_red.jpg renamed!
File 12_red.jpg renamed!
File 13_red.jpg renamed!
File 14_red.jpg renamed!
File 15_red.jpg renamed!
File 16_red.jpg renamed!
File 17_red.jpg renamed!
File 18_red.jpg renamed!
File 19_red.jpg renamed!
File 20_red.jpg renamed!
File 21_red.jpg renamed!
File 22_red.jpg renamed

File 19_green.jpg renamed!
File 20_green.jpg renamed!
File 21_green.jpg renamed!
File 22_green.jpg renamed!
File 23_green.jpg renamed!
File 24_green.jpg renamed!
File 25_green.jpg renamed!
File 26_green.jpg renamed!
File 27_green.jpg renamed!
File 28_green.jpg renamed!
File 29_green.jpg renamed!
File 30_green.jpg renamed!
File 31_green.jpg renamed!
File 32_green.jpg renamed!
File 33_green.jpg renamed!
File 34_green.jpg renamed!
File 35_green.jpg renamed!
File 36_green.jpg renamed!
File 37_green.jpg renamed!
File 38_green.jpg renamed!
File 39_green.jpg renamed!
File 40_green.jpg renamed!
File 41_green.jpg renamed!
File 42_green.jpg renamed!
File 43_green.jpg renamed!
File 44_green.jpg renamed!
File 45_green.jpg renamed!
File 46_green.jpg renamed!
File 47_green.jpg renamed!
File 48_green.jpg renamed!
File 49_green.jpg renamed!
File 50_green.jpg renamed!
File 51_green.jpg renamed!
File 52_green.jpg renamed!
File 53_green.jpg renamed!
File 54_green.jpg renamed!
File 55_green.jpg renamed!
F

File 61_blue.jpg renamed!
File 62_blue.jpg renamed!
File 63_blue.jpg renamed!
File 64_blue.jpg renamed!
File 65_blue.jpg renamed!
File 66_blue.jpg renamed!
File 67_blue.jpg renamed!
File 68_blue.jpg renamed!
File 69_blue.jpg renamed!
File 70_blue.jpg renamed!
File 71_blue.jpg renamed!
File 72_blue.jpg renamed!
File 73_blue.jpg renamed!
File 74_blue.jpg renamed!
File 75_blue.jpg renamed!
File 76_blue.jpg renamed!
File 77_blue.jpg renamed!
File 78_blue.jpg renamed!
File 79_blue.jpg renamed!
File 80_blue.jpg renamed!
File 81_blue.jpg renamed!
File 82_blue.jpg renamed!
File 83_blue.jpg renamed!
File 84_blue.jpg renamed!
File 85_blue.jpg renamed!
File 86_blue.jpg renamed!
File 87_blue.jpg renamed!
File 88_blue.jpg renamed!
File 89_blue.jpg renamed!
File 90_blue.jpg renamed!
File 91_blue.jpg renamed!
File 92_blue.jpg renamed!
File blue renamed!
File 01_green.jpg renamed!
File 02_green.jpg renamed!
File 03_green.jpg renamed!
File 04_green.jpg renamed!
File 05_green.jpg renamed!
File 06_green.

...

File 86_blue.jpg renamed!
File 87_blue.jpg renamed!
File 88_blue.jpg renamed!
File 89_blue.jpg renamed!
File 90_blue.jpg renamed!
File 91_blue.jpg renamed!
File 92_blue.jpg renamed!
File blue renamed!
File 01_green.jpg renamed!
File 02_green.jpg renamed!
File 03_green.jpg renamed!
File 04_green.jpg renamed!
File 05_green.jpg renamed!
File 06_green.jpg renamed!
File 07_green.jpg renamed!
File 08_green.jpg renamed!
File 09_green.jpg renamed!
File 10_green.jpg renamed!
File 11_green.jpg renamed!
File 12_green.jpg renamed!
File 13_green.jpg renamed!
File 14_green.jpg renamed!
File 15_green.jpg renamed!
File 16_green.jpg renamed!
File 17_green.jpg renamed!
File 18_green.jpg renamed!
File 19_green.jpg renamed!
File 20_green.jpg renamed!
File 21_green.jpg renamed!
File 22_green.jpg renamed!
File 23_green.jpg renamed!
File 24_green.jpg renamed!
File 25_green.jpg renamed!
File 26_green.jpg renamed!
File 27_green.jpg renamed!
File 28_green.jpg renamed!
File 29_green.jpg renamed!
File 30_green.jp

File 31_blue.jpg renamed!
File 32_blue.jpg renamed!
File 33_blue.jpg renamed!
File 34_blue.jpg renamed!
File 35_blue.jpg renamed!
File 36_blue.jpg renamed!
File 37_blue.jpg renamed!
File 38_blue.jpg renamed!
File 39_blue.jpg renamed!
File 40_blue.jpg renamed!
File 41_blue.jpg renamed!
File 42_blue.jpg renamed!
File 43_blue.jpg renamed!
File 44_blue.jpg renamed!
File 45_blue.jpg renamed!
File 46_blue.jpg renamed!
File 47_blue.jpg renamed!
File 48_blue.jpg renamed!
File 49_blue.jpg renamed!
File 50_blue.jpg renamed!
File 51_blue.jpg renamed!
File 52_blue.jpg renamed!
File 53_blue.jpg renamed!
File 54_blue.jpg renamed!
File 55_blue.jpg renamed!
File 56_blue.jpg renamed!
File 57_blue.jpg renamed!
File 58_blue.jpg renamed!
File 59_blue.jpg renamed!
File 60_blue.jpg renamed!
File 61_blue.jpg renamed!
File 62_blue.jpg renamed!
File 63_blue.jpg renamed!
File 64_blue.jpg renamed!
File 65_blue.jpg renamed!
File 66_blue.jpg renamed!
File 67_blue.jpg renamed!
File 68_blue.jpg renamed!
File 69_blue

This is save file : C:/Users/user/Melza/UTS/hasil/tokoh/ahmad dahlan\14_ahmaddahlan.jpg
C:/Users/user/Melza/UTS/hasil/tokoh/ahmad dahlan\15_ahmaddahlan.jpg
This is save file : C:/Users/user/Melza/UTS/hasil/tokoh/ahmad dahlan\15_ahmaddahlan.jpg
C:/Users/user/Melza/UTS/hasil/tokoh/ahmad dahlan\16_ahmaddahlan.jpg
This is save file : C:/Users/user/Melza/UTS/hasil/tokoh/ahmad dahlan\16_ahmaddahlan.jpg
C:/Users/user/Melza/UTS/hasil/tokoh/ahmad dahlan\17_ahmaddahlan.jpg
This is save file : C:/Users/user/Melza/UTS/hasil/tokoh/ahmad dahlan\17_ahmaddahlan.jpg
C:/Users/user/Melza/UTS/hasil/tokoh/ahmad dahlan\18_ahmaddahlan.jpg
This is save file : C:/Users/user/Melza/UTS/hasil/tokoh/ahmad dahlan\18_ahmaddahlan.jpg
C:/Users/user/Melza/UTS/hasil/tokoh/ahmad dahlan\19_ahmaddahlan.jpg
This is save file : C:/Users/user/Melza/UTS/hasil/tokoh/ahmad dahlan\19_ahmaddahlan.jpg
C:/Users/user/Melza/UTS/hasil/tokoh/ahmad dahlan\20_ahmaddahlan.jpg
This is save file : C:/Users/user/Melza/UTS/hasil/tokoh/ahmad da

This is save file : C:/Users/user/Melza/UTS/hasil/tokoh/ahmad dahlan\68_ahmaddahlan.jpg
C:/Users/user/Melza/UTS/hasil/tokoh/ahmad dahlan\69_ahmaddahlan.jpg
This is save file : C:/Users/user/Melza/UTS/hasil/tokoh/ahmad dahlan\69_ahmaddahlan.jpg
C:/Users/user/Melza/UTS/hasil/tokoh/ahmad dahlan\70_ahmaddahlan.jpg
This is save file : C:/Users/user/Melza/UTS/hasil/tokoh/ahmad dahlan\70_ahmaddahlan.jpg
C:/Users/user/Melza/UTS/hasil/tokoh/ahmad dahlan\71_ahmaddahlan.jpg
This is save file : C:/Users/user/Melza/UTS/hasil/tokoh/ahmad dahlan\71_ahmaddahlan.jpg
C:/Users/user/Melza/UTS/hasil/tokoh/ahmad dahlan\72_ahmaddahlan.jpg
This is save file : C:/Users/user/Melza/UTS/hasil/tokoh/ahmad dahlan\72_ahmaddahlan.jpg
C:/Users/user/Melza/UTS/hasil/tokoh/ahmad dahlan\73_ahmaddahlan.jpg
This is save file : C:/Users/user/Melza/UTS/hasil/tokoh/ahmad dahlan\73_ahmaddahlan.jpg
C:/Users/user/Melza/UTS/hasil/tokoh/ahmad dahlan\74_ahmaddahlan.jpg
This is save file : C:/Users/user/Melza/UTS/hasil/tokoh/ahmad da

TypeError: catching classes that do not inherit from BaseException is not allowed